# Amazon Augmented AI (A2I) - Amazon Rekognition example

1. [Introduction](#Introduction)
2. [Prerequisites](#Prerequisites)
    1. [Supported Regions](#Supported-Regions)
    2. [Workteam](#Workteam)
    3. [Notebook Permission](#Notebook-Permission)
    4. [Human Loop Role](#Human-Loop-Role)
3. [Client Setup](#Client-Setup)
4. [Sample Data](#Sample-Data)
    1. [Download sample images](#Download-sample-images)
    2. [Upload images to S3](#Upload-images-to-S3)
5. [Create Control Plane Resources](#Create-Control-Plane-Resources)
    1. [Create Human Task UI](#Create-Human-Task-UI)
    2. [Create Flow Definition](#Create-Flow-Definition)
6. [Start Human Loop](#Start-Human-Loop)
    1. [Detect Moderation Labels with AWS Rekognition](#Detect-Moderation-Labels-with-AWS-Rekognition)
    2. [When Human Loop Activation Conditions Are Met](#When-Human-Loop-Activation-Conditions-Are-Met)
    3. [When Human Loop Activation Conditions Are Not Met](#When-Human-Loop-Activation-Conditions-Are-Not-Met)
7. [Monitoring Human Loop](#Monitoring-Human-Loop)
8. [Checking Task Results](#Checking-Task-Results)

### Introduction

In this tutorial, we will be showing you how you can use Amazon Augmented AI (A2I) directly with your calls to Rekognition's Detect Moderation Labels  API. 

For more in depth instructions, visit https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-getting-started.html

To incorporate Amazon A2I into your data labeling workflow for all task types, you need three resources:

* A **worker task template** to create a worker UI. The worker UI displays your input data, such as documents or images, and instructions to workers. It also provides interactive tools that the worker uses to complete your tasks. For more information, see Create a Worker UI.

* A **human review workflow**, also referred to as a flow definition. You use the flow definition to configure your human workforce and provide information about how to accomplish the labeling task. For built-in task types, you also use the flow definition to identify the conditions under which a review human loop is triggered. For example, Amazon Rekognition can perform image content moderation using machine learning. You can use the flow definition to specify that an image will be sent to a human for content moderation review if Amazon Rekognition's confidence is too low. You can create a flow definition in the Amazon SageMaker console or with the Amazon SageMaker API. To learn more about both of these options, see Create a Flow Definition.

* A **human loop** to start your human review workflow. When you use one of the built-in task types, the corresponding AWS service creates and starts a human loop on your behalf when the conditions specified in your flow definition are met or for each object if no conditions were specified. When a human loop is triggered, human review tasks are sent to the workers as specified in the flow definition.

When using a custom task type, you start a human loop using the Amazon Augmented AI Runtime API. When you call StartHumanLoop in your custom application, a task is sent to human reviewers.

### Supported Regions

For the A2I Preview, the **us-east-1** AWS region is supported. Please ensure that any accompanying resources are created in that region, including S3 buckets.

### Workteam or Workforce

A workforce is the group of workers that you have selected to label your dataset. You can choose either the Amazon Mechanical Turk workforce, a vendor-managed workforce, or you can create your own private workforce to label or review your dataset. Whichever workforce type you choose, Amazon SageMaker takes care of sending tasks to workers. 

When you use a private workforce, you also create work teams, a group of workers from your workforce that are assigned to specific jobs— Amazon SageMaker Ground Truth labeling jobs or Amazon Augmented AI human review tasks. You can have multiple work teams and can assign one or more work teams to each job.

To create your Workteam, visit the instructions here: https://docs.aws.amazon.com/sagemaker/latest/dg/sms-workforce-management.html

In [13]:
WORKTEAM_ARN= "<YOUR_WORKTEAM_ARN>"

### Notebook Permission

The AWS IAM Role used to execute the notebook needs to have the following permissions:

* RekognitionFullAccess
* SagemakerFullAccess
* S3 Read Access to the bucket listed below

In [3]:
from sagemaker import get_execution_role

# Setting Role to the default SageMaker Execution Role
ROLE = get_execution_role()
display(ROLE)

'arn:aws:iam::053520186210:role/service-role/AmazonSageMaker-ExecutionRole-20191231T143745'

Visit: https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-permissions-security.html to add the necessary permissions to your role

## Setup
We need to set up the following data:
* `region` - Region to call A2I. Must be us-east-1 for Preview
* `bucket` - A S3 bucket accessible by the given role
    * Used to store the sample images & output results
    * Must be within the same region as the region
* `role` - The IAM role used as part of StartHumanLoop. By default, this notebook will use the execution role
* `workteam` - The Amazon SageMaker Workteam to send the work to. Defaults to the first workteam in the account

## Client Setup

Here we are going to setup the clients. 

In [4]:
import boto3
import io
import json
import uuid
import botocore
import time
import botocore

# Your bucket
BUCKET = '<YOUR_BUCKET>'

# Where your reuslts will be written to. 
OUTPUT_PATH = f's3://{BUCKET}/a2i-results'

REGION = 'us-east-1'

# Amazon SageMaker client
sagemaker = boto3.client('sagemaker', REGION)

# Amazon Rekognition client
rekognition = boto3.client('rekognition', REGION)

# S3 client
s3 = boto3.client('s3', REGION)

# Flow definition name - this value is unique per account and region. You can also provide your own value here.
flowDefinitionName = 'fd-rekognition-demo' 

# Task UI name - this value is unique per account and region. You can also provide your own value here.
taskUIName = 'ui-rekognition-demo'

In [5]:
import pprint

# Pretty print setup
pp = pprint.PrettyPrinter(indent=2)

# Function to pretty-print AWS SDK responses
def print_response(response):
    if 'ResponseMetadata' in response:
        del response['ResponseMetadata']
    pp.pprint(response)

## Sample Data

We'll be using sample images from AWS Rekognition image moderation console.


### Download sample images

In [6]:
# Download images
!wget 'https://dhei5unw3vrsx.cloudfront.net/images/family_picnic_resized.jpg' -O 'family_picnic_resized.jpg'
!wget 'https://dhei5unw3vrsx.cloudfront.net/images/yoga_swimwear_resized.jpg' -O 'yoga_swimwear_resized.jpg'


--2020-03-19 01:43:56--  https://dhei5unw3vrsx.cloudfront.net/images/family_picnic_resized.jpg
Resolving dhei5unw3vrsx.cloudfront.net (dhei5unw3vrsx.cloudfront.net)... 52.85.145.54, 52.85.145.122, 52.85.145.175, ...
Connecting to dhei5unw3vrsx.cloudfront.net (dhei5unw3vrsx.cloudfront.net)|52.85.145.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1009903 (986K) [image/jpeg]
Saving to: ‘family_picnic_resized.jpg’

family_picnic_resiz 100%[===================>] 986.23K  --.-KB/s    in 0.02s   

2020-03-19 01:43:57 (62.0 MB/s) - ‘family_picnic_resized.jpg’ saved [1009903/1009903]

--2020-03-19 01:43:57--  https://dhei5unw3vrsx.cloudfront.net/images/yoga_swimwear_resized.jpg
Resolving dhei5unw3vrsx.cloudfront.net (dhei5unw3vrsx.cloudfront.net)... 52.85.145.213, 52.85.145.54, 52.85.145.122, ...
Connecting to dhei5unw3vrsx.cloudfront.net (dhei5unw3vrsx.cloudfront.net)|52.85.145.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115181 (

Yoga swimwear <img src="yoga_swimwear_resized.jpg" alt="Yoga Swimwear" width="400" />
Family picnic <img src="family_picnic_resized.jpg" alt="Family picnic" width="400" />


### Upload images to S3

Upload the sample images to your S3 bucket. They will be read by AWS Rekognition and by A2I if human task is created.

In [7]:
imageName1 = 'yoga_swimwear_resized.jpg'
imageName2 = 'family_picnic_resized.jpg'

In [8]:
s3.upload_file(imageName1, BUCKET, imageName1)
s3.upload_file(imageName2, BUCKET, imageName2)

You can now check the S3 bucket BUCKET that it contains the images at the specified key paths.

## Create Control Plane Resources


Here we'll be constructing the following control plane resources: task UI and flow definition, using the CreateTaskUI and CreateFlowDefinition APIs, respectively.

These resources can be created once and used to drive any subsequent A2I human loops.

### Create Human Task UI

Create a human task UI resource, giving a UI template in liquid html. This template will be rendered to the a human workers whenever human loop is required.

We are providing a simple demo template that is compatible with AWS Rekognition moderation label input and inferences.

In [9]:
template = r"""
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>
{% capture s3_arn %}http://s3.amazonaws.com/{{ task.input.aiServiceRequest.image.s3Object.bucket }}/{{ task.input.aiServiceRequest.image.s3Object.name }}{% endcapture %}

<crowd-form>
  <crowd-rekognition-detect-moderation-labels
    categories='[
      {% for label in task.input.selectedAiServiceResponse.moderationLabels %}
        {
          name: "{{ label.name }}",
          parentName: "{{ label.parentName }}",
        },
      {% endfor %}
    ]'
    src="{{ s3_arn | grant_read_access }}"
    header="Please select all categories that apply"
  >
    <short-instructions header="Instructions">
      <style>
        .instructions {
          white-space: pre-wrap;
        }
      </style>
      <p class='instructions'>Review the image and choose all applicable categories.
        If no categories apply, choose None.

        <b>Nudity</b>
        Visuals depicting nude male or female person or persons

        <b>Graphic Male Nudity</b>
        Visuals depicting full frontal male nudity, often close ups

        <b>Graphic Female Nudity</b>
        Visuals depicting full frontal female nudity, often close ups

        <b>Sexual Activity</b>
        Visuals depicting various types of explicit sexual activities and pornography

        <b>Illustrated Nudity or Sexual Activity</b>
        Visuals depicting animated or drawn sexual activity, nudity or pornography

        <b>Adult Toys</b>
        Visuals depicting adult toys, often in a marketing context

        <b>Female Swimwear or Underwear</b>
        Visuals depicting female person wearing only swimwear or underwear

        <b>Male Swimwear Or Underwear</b>
        Visuals depicting male person wearing only swimwear or underwear

        <b>Partial Nudity</b>
        Visuals depicting covered up nudity, for example using hands or pose

        <b>Revealing Clothes</b>
        Visuals depicting revealing clothes and poses, such as deep cut dresses

        <b>Graphic Violence or Gore</b>
        Visuals depicting prominent blood or bloody injuries

        <b>Physical Violence</b>
        Visuals depicting violent physical assault, such as kicking or punching

        <b>Weapon Violence</b>
        Visuals depicting violence using weapons like firearms or blades, such as shooting

        <b>Weapons</b>
        Visuals depicting weapons like firearms and blades

        <b>Self Injury</b>
        Visuals depicting self-inflicted cutting on the body, typically in distinctive patterns using sharp objects

        <b>Emaciated Bodies</b>
        Visuals depicting extremely malnourished human bodies

        <b>Corpses</b>
        Visuals depicting human dead bodies

        <b>Hanging</b>
        Visuals depicting death by hanging</p>
    </short-instructions>

    <full-instructions header="Instructions"></full-instructions>
  </crowd-rekognition-detect-moderation-labels>
</crowd-form>
"""

def create_task_ui():
    '''
    Creates a Human Task UI resource.

    Returns:
    struct: HumanTaskUiArn
    '''
    response = sagemaker.create_human_task_ui(
        HumanTaskUiName=taskUIName,
        UiTemplate={'Content': template})
    return response

In [10]:
# Create task UI
humanTaskUiResponse = create_task_ui()
humanTaskUiArn = humanTaskUiResponse['HumanTaskUiArn']
print(humanTaskUiArn)

arn:aws:sagemaker:us-east-1:053520186210:human-task-ui/ui-rekognition-demo


### Create Flow Definition

Now we can create a Flow Definition. The Flow Definition encapsulates the following high-level concepts:

* The AWS managed request source (such as AWS Rekognition content moderation **AWS/Rekognition/Image/ContentModeration/V3**)
* Conditions under which human loop is created, based on the inference result
* Workteam to process the human tasks, number of workers per task etc
* Task UI template to use
* Output S3 location for the human task results

Flow Definition is associated with a particular AWS managed request source, which affects the structure of the human loop activation conditions and format of the inference input and result.

The human loop activation conditions used in this demo are tailored towards AWS Rekognition content moderation - they are based on the confidence thresholds for particular moderation labels.
Activation conditions can be expressed using logical operators *And*, *Or*, *Not*.

In [11]:
def create_flow_definition():
    '''
    Creates a Flow Definition resource

    Returns:
    struct: FlowDefinitionArn
    '''
    humanLoopActivationConditions = json.dumps(
        {
            "Conditions": [
                {
                  "Or": [
                    {
                        "ConditionType": "ModerationLabelConfidenceCheck",
                        "ConditionParameters": {
                            "ModerationLabelName": "Suggestive",
                            "ConfidenceLessThan": 98
                        }
                    },
                    {
                        "ConditionType": "ModerationLabelConfidenceCheck",
                        "ConditionParameters": {
                            "ModerationLabelName": "Female Swimwear Or Underwear",
                            "ConfidenceGreaterThan": 98
                        }
                    }
                  ]
               }
            ]
        }
    )

    response = sagemaker.create_flow_definition(
            FlowDefinitionName= flowDefinitionName,
            RoleArn= ROLE,
            HumanLoopConfig= {
                "WorkteamArn": WORKTEAM_ARN,
                "HumanTaskUiArn": humanTaskUiArn,
                "TaskCount": 1,
                "TaskDescription": "Demo A2I moderation sample task description",
                "TaskTitle": "Demo A2I moderation sample task"
            },
            HumanLoopActivationConfig={
                "HumanLoopRequestSource": {
                    "AwsManagedHumanLoopRequestSource": "AWS/Rekognition/DetectModerationLabels/Image/V3"
                },
                "HumanLoopActivationConditionsConfig": {
                    "HumanLoopActivationConditions": humanLoopActivationConditions
                }
            },
            OutputConfig={
                "S3OutputPath" : OUTPUT_PATH
            }
        )
    print_response(response)
    return response['FlowDefinitionArn'] 


def describe_flow_definition(name):
    '''
    Describes Flow Definition

    Returns:
    struct: response from DescribeFlowDefinition API invocation
    '''
    return sagemaker.describe_flow_definition(
        FlowDefinitionName=name)

In [12]:
# Create a flow definition. We'll be using the flow definition arn for starting human loops.
flow_definition_arn = create_flow_definition()
print(flow_definition_arn)

{ 'FlowDefinitionArn': 'arn:aws:sagemaker:us-east-1:053520186210:flow-definition/fd-rekognition-demo'}
arn:aws:sagemaker:us-east-1:053520186210:flow-definition/fd-rekognition-demo


In [ ]:
# Describe flow definition - status should be active
for x in range(60):
    describeFlowDefinitionResponse = describe_flow_definition(flowDefinitionName)
    print(describeFlowDefinitionResponse['FlowDefinitionStatus'])
    if (describeFlowDefinitionResponse['FlowDefinitionStatus'] == 'Active'):
        print("Flow Definition is active")
        break
    time.sleep(2)

### Detect Moderation Labels with AWS Rekognition

Let's call AWS Rekognition to detect moderation labels on the sample images stored in S3 based on the steps above.

In [ ]:
uniqueId = str(uuid.uuid4())

def detect_moderation_labels(img_name):
    response = rekognition.detect_moderation_labels(Image={'S3Object': {'Bucket': BUCKET, 'Name': img_name}}, 
                                                    HumanLoopConfig={'HumanLoopName': uniqueId + '-1', 'FlowDefinitionArn': flow_definition_arn})
    return response

### When Human Loop Activation Conditions Are Met

Image passed to Rekognition matches the conditions in FlowDefinition to involve Humans. So, HumanLoopArn will be present in the response.

In [ ]:
# The first image has moderation labels, each one with a confidence score
moderationResponse1 = detect_moderation_labels(imageName1)
print_response(moderationResponse1)

### When Human Loop Activation Conditions Are Not Met

Image passed to Rekognition does not match the conditions in FlowDefinition to involve Humans. SO, HumanLoopArn will not be present in the response.

In [ ]:
# The first image has moderation labels, each one with a confidence score
moderationResponse1 = detect_moderation_labels(imageName2)
print_response(moderationResponse1)

In [ ]:
workteamName = WORKTEAM_ARN[WORKTEAM_ARN.rfind('/') + 1:]
print("Navigate to the private worker portal and do the tasks")
print('https://' + sagemaker.describe_workteam(WorkteamName=workteamName)['Workteam']['SubDomain'])

## Monitoring Human Loop

A2I gives the user the ability to monitor the human loop until all the work has been completed by the selected workforce. Using the A2I runtime client, we can check on our human loop and get updates as fast as we need.

In [ ]:
a2i_runtime_client = boto3.client('sagemaker-a2i-runtime', REGION)

describe_human_loop_response = a2i_runtime_client.describe_human_loop(
    HumanLoopName=uniqueId + '-1'
)

display(describe_human_loop_response['HumanLoopStatus'])
display(describe_human_loop_response['HumanLoopOutput'])

## Checking Task Results

Using your private workforce - go ahead and complete the tasks. The results should become available in the S3 OUTPUT_PATH when all work is completed.

The exact path is
`
s3://<OUTPUT_PATH>/<flow-definition-name>/<Year>/<Month>/<Day>/<Minute>/<Hour>/<humanLoopName>/output.json
`

In [ ]:
display(describe_human_loop_response['HumanLoopOutput'])